In [ ]:
"""IMPORTS"""
import numpy as np
from helpers import *
from implementations import *
from utils import *
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

In [ ]:
"""LOADING RAW DATA"""
data_path = 'dataset'
x_tr_raw, x_te_raw, y_tr_raw, train_ids, test_ids = load_csv_data(data_path, sub_sample=False)

In [ ]:
"""REMOVE USELESS COLUMNS"""
columns_to_remove = np.concatenate((range(1, 25), range(54, 56)))

x_tr_remove_col = np.delete(x_tr_raw, columns_to_remove, axis=1)
x_te_remove_col = np.delete(x_te_raw, columns_to_remove, axis=1)

thresh = 1
dup = 5
deg = 5
lambda_ = 0.0002805263157894737

# Treat dk, refuse and nan the same
x_tr_dk = replace_dk_values_with_nan(x_tr_remove_col)
x_val_dk = replace_dk_values_with_nan(x_te_remove_col)

# Nan to mean
x_tr_no_nan, x_val_no_nan = nan_to_mean(x_tr_dk, x_val_dk)

# Standardize
x_tr_std, x_val_std = standardize(x_tr_no_nan, x_val_no_nan)

# Duplicate 1 rows to balance dataset
x_tr_duplicated, y_tr_duplicated = duplicate_1rows(x_tr_std, y_tr_raw, dup)

x_tr_full = x_tr_duplicated
x_val_full = x_val_std
y_tr_full = y_tr_duplicated

In [ ]:
x_poly_tr = build_poly(x_tr_full, deg)
x_poly_val = build_poly(x_val_full, deg)

In [ ]:
tx = np.c_[np.ones((x_poly_tr.shape[0], 1)), x_poly_tr]

y_tr_full_0 = y_tr_full
y_tr_full_0[y_tr_full_0 == -1] = 0

initial_w = np.zeros(tx.shape[1])
max_iters = 1000
gamma = 1e-18

In [ ]:
w_tmp, loss = reg_logistic_regression(y_tr_full_0, tx, lambda_, initial_w, max_iters, gamma)

In [ ]:
y_pred = predict(x_poly_val, w_tmp, "mse")
name = "pen"
create_csv_submission(test_ids, y_pred, name)